In [1]:
import functools
import operator
import psutil
import gensim
from tqdm import tqdm
from transformers import CamembertTokenizer,CamembertModel
from joblib import Parallel,delayed
#import os
#os.chdir("home/jovyan/work")
tok=CamembertTokenizer('MLSUM_tokenizer.model')
camem=CamembertModel.from_pretrained("camembert-base")

W2V=gensim.models.Word2Vec.load("W2V_all.model")
import time
import fats
import pickle

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializin

In [2]:
#test=pickle.load(open('MLSUM_fr_test.pickle','rb'))
#texte=[test.text[:10].values.tolist()[i].split('.') for i in range(10)]

Paragraphes= pickle.load(open('Paragraphes.pickle','rb'))#[i.split('.') for i in test.text.values.tolist()]
Paragraphes=functools.reduce(operator.iconcat, Paragraphes, [])
len(Paragraphes)
#Paragraphes[0]
#resu=fats.Resume(Paragraphes,DL=True,choose_model='SMHA',camem=camem,tok='MLSUM_tokenizer.model')
#fats.Resume(Paragraphes[48*2:49*2],DL=False,cpu=1,type_='TextRankBert',k=2,modele=camem,tok="MLSUM_tokenizer.model",get_score_only=True)


23799

In [3]:
#resu=fats.Resume(Paragraphes,DL=False,cpu=2,type_='TextRankBert',k=2,modele=camem,tok="MLSUM_tokenizer.model",get_score_only=True)
#Paragraphes[251+47]
longueur=[len(i) for i in Paragraphes]
import numpy as np
print(np.max(longueur))
seuil=80
nul=len([i for i in longueur if i>seuil])
print(nul)
print(round(nul/len(Paragraphes)*100,2),"%")

419
347
1.46 %


In [4]:
ME=fats.Make_Embedding(tok=CamembertTokenizer("MLSUM_tokenizer.model"),cpu=1)
input_ids,att_mask=ME.make_token(Paragraphes[251+47],1)
print("OK")

OK


In [ ]:
import torch
c=camem(torch.tensor(input_ids[0:int(len(input_ids)/2)]),
           torch.tensor(att_mask[0:int(len(input_ids)/2)])).last_hidden_state.detach()

In [ ]:
import torch
if len(input_ids)>70: #Au-delà de 70 phrases, camembert plante. On vérifie que le paragraphe en contient moins puis
    #on va découper le paragraphe pour que chaque bout fasse moins de 70
    # d'abord il convient de trouver le chiffre tq len(input_ids)/chiffre<70
    for i in range(2,100):
        if (len(input_ids)/i)<70:
            h=i
            print(h)
            break
        else:
            continue
    #une fois qu'on a ce chiffre h on y va :
    embeddings=[]
    for i in range(h):
        x_1=int(len(input_ids)*(i/h))
        x_2=int(len(input_ids)*((i+1)/h))
        print(x_1,x_2)
        embeddings.append(camem(torch.tensor(input_ids[x_1:x_2]),
           torch.tensor(att_mask[x_1:x_2])).last_hidden_state.detach())
    embeddings=torch.cat(embeddings)
    

In [10]:
torch.cat(embeddings).size()

torch.Size([85, 512, 768])

In [ ]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(figsize=(16,12))
ax.hist(longueur)

In [3]:
MEB=fats.TextRank("MLSUM_tokenizer.model",cpu=1).make_embedding_bert
MEB(Paragraphes[251+47],camem)

(tensor([[ 0.0317,  0.1234,  0.0652,  ..., -0.0192, -0.0331, -0.1243],
         [-0.0206,  0.2380,  0.0225,  ..., -0.0392,  0.0044, -0.1620],
         [-0.0927,  0.1113,  0.0838,  ...,  0.0675, -0.0221, -0.1592],
         ...,
         [-0.0286,  0.0901,  0.1715,  ...,  0.0044, -0.0259, -0.1014],
         [-0.0585,  0.1646,  0.0774,  ..., -0.0571, -0.0480, -0.1135],
         [-0.0290,  0.1889,  0.0935,  ...,  0.0064, -0.0407, -0.1483]]),
 {})

In [4]:
#s=time.time()
ms=fats.BERTScore(tok="MLSUM_tokenizer.model",cpu=1).make_score
#print(ms(Paragraphes[251+47]))
#bs.make_score(Paragraphes[251+47])
#(time.time()-s)/60

In [5]:
rendu=pickle.load(open('BSR_sortie.pickle','rb'))#[] if 
rendu[0]

tensor([1, 7, 2])

In [ ]:
rendu=pickle.load(open('BSR_sortie.pickle','rb'))#[] if 
print(len(rendu))
ms=fats.BERTScore(tok="MLSUM_tokenizer.model",cpu=1).make_score

#pas=1
#s
for i in tqdm(range(len(rendu),len(Paragraphes))):
    resu=ms(Paragraphes[i])
    #fats.Resume(Paragraphes[(i*2):(i+pas)*2],DL=False,cpu=1,type_='TextRankBert',k=2,modele=camem,tok="MLSUM_tokenizer.model",get_score_only=True)
    rendu.append(resu)
               
    if i%50==0:
        pickle.dump(rendu,open('BSR_sortie.pickle','wb'))

651


  2%|▏         | 393/23148 [13:10<16:22:27,  2.59s/it]

In [ ]:
fats.make_U_resume(Paragraphes,cpu=1,type_='TextRankBert',k=2,modele=camem)

In [4]:
len(rendu)

11899

In [ ]:
rendu_2=functools.reduce(operator.iconcat, rendu, [])
len(rendu_2)

In [ ]:
rendu[:2]